In [2]:
import os
import requests
import pendulum
import dotenv

dotenv.load_dotenv()

def github_api_request(type, url, last_fetch_at, params=None):

    token = os.getenv("GITHUB_TOKEN")
    if not token:
        raise ValueError("Github token is not set")

    headers = {"Authorization": f"Bearer {token}"}

    if last_fetch_at:
        headers["if-modified-since"] = last_fetch_at.strftime(
            "%a, %d %b %Y %H:%M:%S GMT"
        )

    if type == "GET":
        response = requests.get(url, headers=headers, params=params)
    elif type == "POST":
        response = requests.post(url, headers=headers, json=params)
    elif type == "PATCH":
        response = requests.patch(url, headers=headers, json=params)
    elif type == "PUT":
        response = requests.put(url, headers=headers, json=params)
    elif type == "DELETE":
        response = requests.delete(url, headers=headers)
    else:
        raise ValueError(f"Unsupported type: {type}")

    if response.status_code == 200:
        # print(f"==> 200 OK for {url}")
        pass
    elif response.status_code == 304:
        print(f"==> 304 Not Modified since the last fetch: {url}")
        pass
    elif response.status_code == 403:
        print(f"403 Forbidden Error for {url} / Message: {response.text}")
        print(
            f"remaining rate limit: {response.headers['X-RateLimit-Remaining']} reset: {pendulum.from_timestamp(int(response.headers['X-RateLimit-Reset'])).in_tz('America/Montreal')}"
        )
        pass
    elif response.status_code == 404:
        print(f"Not Found:{url}")
        pass
    else:
        print(f"{response.status_code} Error for {url} / Message: {response.text}")
        pass

    return response

In [3]:
from collections import defaultdict


def build_tree(paths):
    tree = lambda: defaultdict(tree)
    root = tree()
    for path in paths:
        parts = path.split("/")
        current_level = root
        for part in parts:
            current_level = current_level[part]
    return root


def tree_to_string(tree, indent=""):
    tree_str = ""
    for key, subtree in sorted(tree.items()):
        tree_str += f"{indent}├── {key}\n"
        if subtree:
            tree_str += tree_to_string(subtree, indent + "│ ")
    return tree_str


def generate_tree(paths):
    tree = build_tree(paths)
    return tree_to_string(tree)

In [4]:
response = github_api_request(
    "GET",
    "https://api.github.com/repos/ahmed-moubtahij/TokenHealer/git/trees/d2b700230d85a0fac0728eaffc27a25d7642f921",
    last_fetch_at=None,
    params={"recursive": "true"},
)
data = response.json()
tree = data["tree"]
tree_text = generate_tree([item["path"] for item in tree])
print(f"==>> tree_text: {tree_text}")

==>> tree_text: ├── .gitignore
├── LICENSE
├── README.md
├── example.py
├── poetry.lock
├── pyproject.toml
├── test
│ ├── test_token_healing.py
│ ├── test_trie.py
├── tokenhealing
│ ├── __init__.py
│ ├── token_healing.py
│ ├── trie.py

